Формируем данные

In [ ]:
import os
import re 

def get_file_tokens(path):
    tokens = []
    with open(path, "r") as f:
        line = f.readline()
        while line:
            if line:
                if line!='\n':
                    token = line.split('\t')
                    t=token[len(token)-1]
                    tokens.append(t[:len(t)-1])
            line = f.readline()
    f.close()
    return tokens

In [ ]:
import os
import glob

def GetAllTokens(rootDir):
    tokens=[]
    for lists in os.listdir(rootDir):
                    path = os.path.join(rootDir, lists)
                    for filename in glob.glob(path+'\\*'):
                       tokens = tokens + get_file_tokens(filename)
    return tokens

In [ ]:
lemmas = GetAllTokens('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\annotated-corpus\\train\\')

In [ ]:
lemmas+=GetAllTokens('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\annotated-corpus\\test\\')

In [ ]:
def write_dict(path,t):
    with open(path, "w") as f:
        for i in t:
            f.write(i)
            f.write('\n')
    f.close()

In [ ]:
write_dict("lemmas_p.txt",lemmas)

Удаление знаков пунктуации

In [ ]:
import re

lemm=[]
for l in lemmas:
    if not re.match('\.|\?|\,|\:|\!|\<|\>|\-|\'|\"|\:|\(|\)|\;',l):
        lemm.append(l)


In [ ]:
lemm

Приведение к нижнему регистру

In [ ]:
lem=[]
for l in lemm:
    lem.append(l.lower())

In [ ]:
lem

Удаление стоп слов

In [ ]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

lm=[]
for l in lem:
	if l not in stop_words:
		lm.append(l)

In [ ]:
lm

In [ ]:
write_dict("lemmas.txt",lm)

Извлечение триграмм и подсчет их частот

In [ ]:
def read_dict(path):
    dictionary = []
    with open(path, "r") as f:
        while True:
            line=f.readline()
            if not line:
                break
            dictionary.append(line.replace('\n',''))
    return dictionary

In [ ]:
data=read_dict("lemmas.txt")

In [ ]:
data

In [ ]:
threegrams = []
for i in range(len(data) - 2):
        threegrams.append((data[i], data[i+1], data[i+2]))

In [ ]:
threegrams

In [ ]:
freqs_treegram= dict()
for threegram in threegrams:
    freqs_treegram[threegram] = freqs_treegram.get(threegram, 0) + 1

In [ ]:
freqs_treegram

In [ ]:
freqs= dict()
for d in data:
    freqs[d] = freqs.get(d, 0) + 1

In [ ]:
freqs

Алгоритм расчета меры ассоциации: лексемы, MI (вариант 3)

In [ ]:
from math import pow
import math

def mi(threegram, freqs_treegram,freqs, N):
    ngram = 3
    return math.log((freqs_treegram[threegram]/(freqs[threegram[0]]*freqs[threegram[1]]*freqs[threegram[2]]))*pow(N, ngram - 1),2)


In [ ]:
mi_measures=dict()
for threegram in threegrams:
    mi_measures[threegram]=mi(threegram=threegram,freqs_treegram=freqs_treegram,freqs=freqs,N=len(data))


In [ ]:
mi_measures

In [ ]:
mi_measures_sort=dict(sorted(mi_measures.items(), key=lambda item: item[1], reverse=True))

In [ ]:
from collections import Counter

threegram_top=Counter(mi_measures).most_common(30)

In [ ]:
threegram_top

In [ ]:
with open("threegram_top.txt", "w") as f:
    for key in mi_measures_sort:
        f.write(str(key))
        f.write(str(mi_measures_sort[key]))
        f.write('\n')
f.close()

Проверка

In [ ]:
import nltk
from nltk.collocations import *


trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(data)

for i in finder.score_ngrams(trigram_measures.pmi):
    print(i)